In [ ]:
import json
import pandas as pd
import shapely
import geopandas as gpd
from shapely.geometry import LineString, Point
import pandana
import osmnet

In [ ]:
lat_min = 42.904155
long_min = -2.621987
lat_max = 43.403070
long_max = -1.740334

## Drive

In [ ]:
drive_nodes, drive_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max,
    lng_max=long_max, network_type='drive', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 12,901.5KB from www.overpass-api.de in 2.11 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 9,842.5KB from www.ove

In [ ]:
drive_nodes=drive_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [ ]:
# Add lat and long to each edge

drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [ ]:

drive_edges['geometry']=drive_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [ ]:
# Create geodataframe

drive_edges_gdf=gpd.GeoDataFrame(data=drive_edges, geometry='geometry', crs='epsg:4326')
drive_edges_gdf

from           to    distance  \
833859      301662479         833859    301662479   20.132225   
301662479   833859         301662479       833859   20.132225   
5462312     299850007        5462312    299850007   56.146191   
299850007   5462312        299850007      5462312   56.146191   
            299850058      299850007    299850058  368.407367   
...                              ...          ...         ...   
11554815339 11554815340  11554815339  11554815340   13.390456   
2424498315  11601681881   2424498315  11601681881   63.136354   
11601681881 2424498315   11601681881   2424498315   63.136354   
            11601681882  11601681881  11601681882    5.178593   
11601681882 11601681881  11601681882  11601681881    5.178593   

                                     name    ref       highway service bridge  \
833859      301662479                 NaN  A-636      motorway     NaN    yes   
301662479   833859                    NaN  A-636      motorway     NaN    yes   
5462312     299850007                 NaN    NaN    trunk_link     NaN    NaN   
299850007   5462312                   NaN    NaN    trunk_link     NaN    NaN   
            299850058                 NaN    NaN    trunk_link     NaN    NaN   
...                                   ...    ...           ...     ...    ...   
11554815339 11554815340  Larramendi kalea    NaN   residential     NaN    NaN   
2424498315  11601681881       Elduainalde    NaN  unclassified     NaN    NaN   
11601681881 2424498315        Elduainalde    NaN  unclassified     NaN    NaN   
            11601681882       Elduainalde    NaN  unclassified     NaN    yes   
11601681882 11601681881       Elduainalde    NaN  unclassified     NaN    yes   

                        tunnel access  ... area width    junction  from_lon  \
833859      301662479      NaN    NaN  ...  NaN   NaN         NaN -2.216077   
301662479   833859         NaN    NaN  ...  NaN   NaN         NaN -2.216319   
5462312     299850007      NaN    NaN  ...  NaN   NaN         NaN -2.462793   
299850007   5462312        NaN    NaN  ...  NaN   NaN         NaN -2.463082   
            299850058      NaN    NaN  ...  NaN   NaN         NaN -2.463082   
...                        ...    ...  ...  ...   ...         ...       ...   
11554815339 11554815340    NaN    NaN  ...  NaN   NaN  roundabout -2.071467   
2424498315  11601681881    NaN    NaN  ...  NaN   NaN         NaN -1.988407   
11601681881 2424498315     NaN    NaN  ...  NaN   NaN         NaN -1.987796   
            11601681882    NaN    NaN  ...  NaN   NaN         NaN -1.987796   
11601681882 11601681881    NaN    NaN  ...  NaN   NaN         NaN -1.987748   

                          from_lat         id_x    to_lon     to_lat  \
833859      301662479    43.044569       833859 -2.216319  43.044609   
301662479   833859       43.044609    301662479 -2.216077  43.044569   
5462312     299850007    43.076158      5462312 -2.463082  43.076616   
299850007   5462312      43.076616    299850007 -2.462793  43.076158   
            299850058    43.076616    299850007 -2.464266  43.079813   
...                            ...          ...       ...        ...   
11554815339 11554815340  43.145538  11554815339 -2.071629  43.145559   
2424498315  11601681881  43.130347   2424498315 -1.987796  43.130697   
11601681881 2424498315   43.130697  11601681881 -1.988407  43.130347   
            11601681882  43.130697  11601681881 -1.987748  43.130728   
11601681882 11601681881  43.130728  11601681882 -1.987796  43.130697   

                                id_y  \
833859      301662479      301662479   
301662479   833859            833859   
5462312     299850007      299850007   
299850007   5462312          5462312   
            299850058      299850058   
...                              ...   
11554815339 11554815340  11554815340   
2424498315  11601681881  11601681881   
11601681881 2424498315    2424498315   
            11601681882  11601681882   
11601681882 11601681881 

### Ensure all roads have valid speed

In [ ]:
# Delete "_link"

drive_edges_gdf['highway']=drive_edges_gdf['highway'].apply(lambda hw: hw.replace('_link', ''))

# Fill null values
# Then, change from "60 km/h" to "60"

drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].fillna(0)
drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].apply(lambda s: int(str(s).split(' ')[0]))

# Group highways with type and save mode for each type

grouped = drive_edges_gdf.loc[drive_edges_gdf['maxspeed']>0].groupby('highway')['maxspeed'].agg(pd.Series.mode)
for t in drive_edges_gdf.highway.unique():
    if t not in grouped.keys():
        grouped[t] = grouped["unclassified"]

# Replace zero values with group mode
drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'maxspeed'] = drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'highway'].apply(lambda x: grouped[x])

In [ ]:
# Add m/s speed and time as a function of distance and speed.

drive_edges_gdf['speed_m_s']=0.8*drive_edges_gdf['maxspeed']*1000/3600
drive_edges_gdf['drive_time_s']=drive_edges_gdf['distance']/drive_edges_gdf['speed_m_s']

## Create Shapefile for GAMA

In [ ]:
# drive_edges_gdf.to_file(f"data/drive_shapefile.shp", driver='ESRI Shapefile')

### Create pandana network

In [ ]:
drive_net_pdna=pandana.network.Network(
    drive_nodes['lon'], drive_nodes['lat'],
    drive_edges_gdf['from'], drive_edges_gdf['to'],
    drive_edges_gdf[['distance','speed_m_s','drive_time_s']], twoway=False)

In [ ]:
drive_net_pdna.save_hdf5("../input_data/networks/drive_net.h5")

In [ ]:
drive_nodes.to_csv("../input_data/networks/drive_nodes.csv")

In [ ]:
# drive_edges_gdf.to_file('../data/drive_net.geojson', index=False)
# drive_edges_gdf.to_file('../front_end/data/drive_net.geojson', index=False)
# drive_nodes.to_csv('../data/drive_nodes.csv')

## Walk and Bike

In [ ]:
walk_nodes, walk_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max,
    lng_max=long_max, network_type='walk', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 44,734.0KB from www.overpass-api.de in 2.22 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 30,778.0KB from www.overpass-api.de in 1.76 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

In [ ]:
walk_nodes=walk_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [ ]:
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [ ]:
walk_edges['geometry']=walk_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [ ]:
walk_edges_gdf=gpd.GeoDataFrame(data=walk_edges, geometry='geometry', crs='epsg:4326')

In [ ]:
walk_nodes.to_csv("../input_data/networks/walk_nodes.csv")

### Create walk pandana network

In [ ]:
walk_net_pdna=pandana.network.Network(
    walk_nodes['lon'], walk_nodes['lat'],
    walk_edges_gdf['from'], walk_edges_gdf['to'],
    walk_edges_gdf[['distance']], twoway=False)

In [ ]:
walk_net_pdna.save_hdf5("../input_data/networks/walk_net.h5")

In [ ]:
# walk_edges_gdf.to_file('../data/walk_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/walk_net.geojson', index=False)
# walk_nodes.to_csv('../data/walk_nodes.csv')

### Create bike pandana network

In [ ]:
walk_net_pdna.save_hdf5("../input_data/networks/bike_net.h5")

In [ ]:
# walk_edges_gdf.to_file('../data/bike_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/bike_net.geojson', index=False)
# walk_nodes.to_csv('../data/bike_nodes.csv')